In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, root_scalar
from scipy.special import factorial
import scipy.stats as stats
import paramiko
from getpass import getpass
from array import array
import pickle
%matplotlib inline
import os

In [3]:
class Singles:
    def __init__(self, eventBytes,compact = True, keephighestenergy = False, delay = False, savetxt = False):
    
        self.frame = self.GetTimeFrame(eventBytes,compact)
        self.channelid = self.GetChannelID(eventBytes)
        self.submoduleid = self.GetSubmoduleID(eventBytes)
        self.moduleid = self.GetModuleID(eventBytes,compact)
        self.valid = self.IsValid(eventBytes)
        eventBytes = eventBytes[self.valid, ...]
        self.frame = self.frame[self.valid,...]
        self.keep = self.valid[self.valid, ...]
        self.size = self.keep.size
        self.valid = self.valid[self.valid, ...]
        self.coarse = self.GetCoarseTime(eventBytes)
        self.moduleid = self.GetModuleID(eventBytes,compact)
        self.subframe = self.Getsubframe(eventBytes,compact)
        self.submoduleid = self.GetSubmoduleID(eventBytes)
        self.halfchipid = self.GetHalfChipID(eventBytes)
        self.channelid = self.GetChannelID(eventBytes)
        self.energy = self.GetEnergy(eventBytes)
        self.crystalID = self.GetCrystalId()
        self.fine = self.GetFineTime(eventBytes)
        self.moduleid = self.GetModuleID(eventBytes,compact)
        self.index = np.linspace(0, self.size - 1, self.size, dtype = np.int64)
        if keephighestenergy:
            print(self.coarse[:20])
            print(self.energy[:20])
            argsort = np.lexsort((self.energy*(-1), self.coarse, self.frame))
            eventBytes = eventBytes[argsort, ...]
            self.frame = self.frame[argsort,...]
            self.coarse = self.GetCoarseTime(eventBytes)
            
            
            print(self.coarse[:20])
            print(self.energy[:20])
            
            valid = (np.diff(self.coarse) > 10)
            valid = np.insert(valid, 0, True)
            eventBytes = eventBytes[valid, ...]
            self.frame = self.frame[valid,...]
            
            self.channelid = self.GetChannelID(eventBytes)
            self.submoduleid = self.GetSubmoduleID(eventBytes)
            self.moduleid = self.GetModuleID(eventBytes,compact)
            self.valid = self.IsValid(eventBytes)
            eventBytes = eventBytes[self.valid, ...]
            self.keep = self.valid[self.valid, ...]
            self.size = self.keep.size
            self.frame = self.frame[self.valid,...]
            self.moduleid = self.GetModuleID(eventBytes,compact)
            self.subframe = self.Getsubframe(eventBytes,compact)
            self.submoduleid = self.GetSubmoduleID(eventBytes)
            self.halfchipid = self.GetHalfChipID(eventBytes)
            self.channelid = self.GetChannelID(eventBytes)
            self.energy = self.GetEnergy(eventBytes)
            self.crystalID = self.GetCrystalId()
            self.fine = self.GetFineTime(eventBytes)
            self.fine_ps = self.GetFinepsTime(eventBytes)
            self.coarse = self.GetCoarseTime(eventBytes)
            self.index = np.linspace(0, self.size - 1, self.size, dtype = np.int64)
            #print(self.frame)
        self.moduleid = self.GetModuleID(eventBytes,compact)
        self.index = np.linspace(0, self.size - 1, self.size, dtype = np.int64)
        if savetxt:
            with open('data.txt', 'w') as f:
                for i in range(int(self.index.shape[0]/2)):
                    f.write("{:08x}".format(self.coarse[i*2+1]) +" "+  "{:08x}".format(self.coarse[i*2]) +" "+"{:08x}".format(self.index[i*2+1]) +" "+ "{:08x}".format(self.index[i*2]))
                    f.write("\n")
            f.close()
    def IsValid(self, _eventBytes):
        hit_Master_testHit = _eventBytes[:, 2] & 0b11100000
        bad = _eventBytes[:, 4] & 0b10000000
        valid1 = (hit_Master_testHit == 192)
        valid2 = (self.moduleid < 16)
        valid3 = (self.channelid < 18)
        valid4 = (self.submoduleid < 6)
        valid5 = (bad == 128)
        #valid_tmp = (self.moduleid == 0) | (self.moduleid == 8)
        #return valid2&valid4
        return valid1 & valid2 & valid3 & valid4 & valid5
    def GetChannelID(self, _eventBytes):
        channelID = _eventBytes[:, 2] & 0b00011111
        #return _eventBytes[:, 2]
        return channelID
    def GetEnergy(self, _eventBytes):
        bit8 = np.uint16(_eventBytes[:, 5] & 0b10000000) << 1
        bit7_0 = _eventBytes[:, 7]
        #return bit7_0
        return bit8 + bit7_0
    def GetModuleID(self, _eventBytes,compact):
        if compact:
            ModuleID = _eventBytes[:, 1] & 0b00001111
        else:
            ModuleID = (_eventBytes[:, 0] & 0b11000000) >> 6
        return ModuleID
    def GetSubmoduleID(self, _eventBytes):
        submoduleID = (_eventBytes[:, 0] & 0b00111000) >> 3
        return submoduleID
    def GetHalfChipID(self, _eventBytes):
        halfChipID = _eventBytes[:, 0] & 0b00000111
        return halfChipID
    def Getsubframe(self, _eventBytes, compact):
        if not compact:
            subframe = _eventBytes[:, 1]
        else:
            subframe = (_eventBytes[:, 1] & 0b11110000) >> 4
        return subframe   
    def GetTimeFrame(self, _eventBytes, compact):
        frame = (np.uint32(_eventBytes[:, 4]) << 8*0) + (np.uint32(_eventBytes[:, 5]) << 8*1) + (np.uint32(_eventBytes[:, 6]) << 8*2) + np.uint32(_eventBytes[:, 7] << 8*3)
        frame[_eventBytes[:,0] != 0xfa] = 0
        pos = np.where(frame > 0)[0]
        prev_pos = pos[0]
        for i in pos[1:]:
            frame[prev_pos:i] = frame[prev_pos]
            prev_pos = i
        frame[pos[-1]:] = frame[pos[-1]]
        return frame
    def GetFineTime(self, _eventBytes):
        finepsTime_9_8 = np.uint16(_eventBytes[:, 4] & 0b01100000) << 3
        finepsTime_7_0 = _eventBytes[:, 3]
        return finepsTime_9_8 + finepsTime_7_0
    def GetCoarseTime(self, _eventBytes):
        coarseBit_14_8 = np.uint16(_eventBytes[:, 5] & 0b01111111) << 8
        coarseBit_7_0 = _eventBytes[:, 6]
        return coarseBit_14_8 + coarseBit_7_0
    def GetCrystalId(self):
        # get (flatten) crystal ID from (module ID * 6 * 8 * 18 + submoduleID *
        # 8 * 18 + HalfChipID * 18 + channelID):
        return np.uint16(self.moduleid) * 864 + np.uint16(self.submoduleid) * 144 + np.uint16(self.halfchipid) * 18 + self.channelid
    def GetKeepEvents(self, keep_index):
        self.size = len(keep_index)
        self.keep = self.keep[keep_index]
        self.energy = self.energy[keep_index]
        self.moduleid = self.moduleid[keep_index]
        self.submoduleid = self.submoduleid[keep_index]
        self.halfchipid = self.halfchipid[keep_index]
        self.channelid = self.channelid[keep_index]
        self.crystalID = self.crystalID[keep_index]
        self.frame = self.frame[keep_index]
        self.fine = self.fine[keep_index]
        self.coarse = self.coarse[keep_index]
        self.index = keep_index
        return self
    def AddKeepEvents(self, event_keep):
        if event_keep == []:
            return
        self.size = self.size + event_keep.size
        self.keep = np.concatenate((self.keep, event_keep.keep))
        self.energy = np.concatenate((self.energy, event_keep.energy))
        self.moduleid = np.concatenate((self.moduleid, event_keep.moduleid))
        self.submoduleid = np.concatenate((self.submoduleid, event_keep.submoduleid))
        self.halfchipid = np.concatenate((self.halfchipid, event_keep.halfchipid))
        self.channelid = np.concatenate((self.channelid, event_keep.channelid))
        self.crystalID = np.concatenate((self.crystalID, event_keep.crystalID))
        self.frame = np.concatenate((self.frame, event_keep.frame))
        self.fine = np.concatenate((self.fine, event_keep.fine))
        self.coarse = np.concatenate((self.coarse, event_keep.coarse))
        self.index = np.linspace(0, self.size - 1, self.size, dtype = np.int64)
        return self

In [4]:
CYCLE = 1.6e-9  # clock cycle (s)
TAU = 3 * CYCLE  # coincidence window (s)
DELAY = 10 * CYCLE  # delay for DW estimate (s)
DETECTORS = 48 * 48
FILE_RANGE = range(1, 100 + 1)

In [5]:
def load_file(n):
    with open(f'C:/scratch/singles_{n}.pkl', 'rb') as f:
        results = pickle.load(f)
    return results

In [6]:
results = load_file(1)
results2 = load_file(15)

In [8]:
len(np.unique(results.moduleid))

16

In [9]:
len(np.unique(results.submoduleid))

6

In [7]:
singles = pd.DataFrame()

In [8]:
singles['time'] = results.coarse * 1.6e-9 + results.fine * 50e-12
singles['detector'] = results.crystalID
singles['energy'] = results.energy

In [9]:
times = np.array(singles['time'])

In [9]:
import ctypes

In [ ]:
with open('times.txt', 'w') as f:
    f.writelines(list(singles.astype('string')['time']))

In [39]:
TIME = max(singles['time'])

In [46]:
def bundle_coincidences(singles):
    """
        Bundle singles into coincidences, returns a DataFrame with columns:
        time1, time2, detector1, detector2, source1, source2, true
        where true is True if the two singles are from the same source, (true coincidence)
        False otherwise
    """
    coins = []
    window_start = 0
    possible_coincidence = []
    for i in range(0, len(singles['time']) - 1):
        if i % 100000 == 0:
            print(f"Processing singles {i}/{len(singles['time']) - 1}")
        if singles['time'][i] - window_start >= TAU:
            # process any previous coincidences
            if len(possible_coincidence) >= 2:
                # If there are at least 2 singles in the window, we have a possible coincidence
                if len(possible_coincidence) > 2:
                    energies = [singles['energy'][j] for j in possible_coincidence]
                    energies = dict(zip(energies, possible_coincidence))
                    true_coinc = (energies.pop(max(energies)), energies.pop(max(energies)))
                    # Take just the two highest energies
                    # takeWinnerOfGoods policy
                else:
                    true_coinc = tuple(possible_coincidence)
                coins.append(true_coinc)

            # Reset the window
            possible_coincidence = [i]
            window_start = singles['time'][i]
        elif singles['time'][i] - window_start < TAU:
            possible_coincidence.append(i)  # Add to coincidence list
    coinci = pd.DataFrame()
    coinci['time1'] = [singles['time'][i[0]] for i in coins]
    coinci['time2'] = [singles['time'][i[1]] for i in coins]
    coinci['detector1'] = [singles['detector'][i[0]] for i in coins]
    coinci['detector2'] = [singles['detector'][i[1]] for i in coins]
    # coinci['source1'] = [singles['source'][i[0]] for i in coins]
    # coinci['source2'] = [singles['source'][i[1]] for i in coins]
    # coinci['true'] = coinci['source1'] == coinci['source2']
    return coinci


def sp_counts(singles, coincidences):
    """
        Calculate Singles-Prompts (SP) counts for singles and coincidences
        Returns a tuple of two dicts: singles_count, prompts_count
        singles_count is a dict of form {detectorID: count} for singles
        prompts_count is a dict of form {detectorID: count} for prompts
        where prompts are the sum of all coincidences involving a given detector
    """
    # Calculate singles/prompts counts
    det1_counts = coincidences['detector1'].value_counts().to_dict()  # det1 coincidences involved
    det2_counts = coincidences['detector2'].value_counts().to_dict()  # det2 coincidences involved

    # Prompts are the sum of all coincidences involving a given detector,
    # regardless of first or second
    # Every detector, regardless of presence, has a key
    prompts = pd.DataFrame({'detector': list(range(DETECTORS))})
    prompts['prompts'] = prompts['detector'].map(
        lambda x: det1_counts.get(x, 0) +
        det2_counts.get(x, 0)
    )

    # dicts of form {detectorID: count}
    prompts_count = prompts.set_index('detector')['prompts'].to_dict()
    singles_count = singles['detector'].value_counts().to_dict()

    return singles_count, prompts_count


def randomsrate(i, j, singles_count, prompts_count, L, S):
    """
        Returns the Singles-Prompts (SP) randoms rate estimate from a pair
        of detectors with crystalIDs i and j per Oliver & Rafecas
    """
    # P_i and P_j are prompts rates for detectors i, j
    P_i = prompts_count.get(i, 0) / TIME  # .get(i, 0) returns 0 if det i not in prompts_count
    P_j = prompts_count.get(j, 0) / TIME
    # S_i and S_j are singles rates for detectors i, j
    S_i = singles_count.get(i, 0) / TIME
    S_j = singles_count.get(j, 0) / TIME
    # Coefficient for the randoms rate equation
    coeff = (2 * TAU * np.exp(-(L + S)*TAU))/((1 - 2 * L * TAU)**2)
    # Further terms in SP equation
    i_term = S_i - np.exp((L + S)*TAU) * P_i
    j_term = S_j - np.exp((L + S)*TAU) * P_j
    return coeff * i_term * j_term


def singles_prompts(singles_count, prompts_count, singles, coincidences):
    """
        Calculate Singles-Prompts (SP) estimate of total randoms rate
    """

    S = len(singles) / TIME  # Rate of singles measured by scanner as a whole
    P = 2 * len(coincidences) / TIME  # Twice the prompts rate

    # Roots of this function are the lambda (L) values needed for the SP estimate.
    def lambda_eq(L):
        return 2 * TAU * L * L - L + S - P * np.exp((L + S)*TAU)
    L = root_scalar(lambda_eq, x0=0)
    if not L.converged:
        raise RuntimeError("Failed to converge on lambda.")
    L = L.root

    # Calculate the Singles-Prompts rate estimate for the whole scanner
    # summing over all pairs of detectors
    sp_rate = 0
    for i in range(DETECTORS):
        for j in range(i, DETECTORS):
            sp_rate += randomsrate(i, j, singles_count, prompts_count, L, S)
    return sp_rate * TIME


# Calculate delayed-window estimate of total randoms rate
def delayed_window(singles):
    dw_estimate = 0
    for t in singles['time']:
        dw_estimate += (
            np.searchsorted(singles['time'], t + DELAY + TAU) -
            np.searchsorted(singles['time'], t + DELAY)
        )  # Num of singles in delayed window
    return dw_estimate


# Calculate singles-rate estimate of total randoms rate
def singles_rate(singles_count):
    sr_rate = 0
    for i in range(DETECTORS):
        for j in range(i, DETECTORS):
            sr_rate += 2 * TAU * singles_count.get(i, 0) / TIME * singles_count.get(j, 0) / TIME
    return sr_rate * TIME

In [47]:
coins = bundle_coincidences(singles)

Processing singles 0/63790025
Processing singles 100000/63790025
Processing singles 200000/63790025
Processing singles 300000/63790025
Processing singles 400000/63790025
Processing singles 500000/63790025
Processing singles 600000/63790025
Processing singles 700000/63790025
Processing singles 800000/63790025
Processing singles 900000/63790025
Processing singles 1000000/63790025
Processing singles 1100000/63790025
Processing singles 1200000/63790025
Processing singles 1300000/63790025
Processing singles 1400000/63790025
Processing singles 1500000/63790025
Processing singles 1600000/63790025


KeyboardInterrupt: 

In [12]:
np.unique(results.crystalID)

array([    0,     1,     2, ..., 13821, 13822, 13823], dtype=uint16)

In [ ]:
singles_count, prompts_count = sp_counts(singles, coins)

In [ ]:
singles_prompts(singles_count, prompts_count, singles, coins)

In [ ]:
delayed_window(singles)

In [ ]:
singles_rate(singles_count)